In [ ]:
project_root = 'temp/'

video_path = '../demo/case2-openfield/openfield-1min-raw.mp4'
frame0_mask_path = f'{project_root}frame0_mask.npy'

DEVICE = 'cuda' # cuda, mps
MODEL = 'vit_b'
crf = 18

In [ ]:
# Video segmentation
import numpy as np
from tqdm import tqdm
from castle import generate_aot
from castle.utils.plot import generate_mix_image, generate_mask_image
from castle.utils.video_io import ReadArray, WriteArray
MODEL = 'r50_deaotl'
tracker = generate_aot(model_type=MODEL, device=DEVICE)

mask_full = np.load(frame0_mask_path)
num_object = np.max(mask_full) # 1
print('num_object:', num_object)
frame0 = ReadArray(video_path)[0]
tracker.add_reference_frame(frame0, mask_full, num_object)

In [ ]:
out1_path = f'{project_root}mix.mp4'
out2_path = f'{project_root}mask.mp4'

video = ReadArray(video_path)
out1 = WriteArray(out1_path, video.fps, crf)
out2 = WriteArray(out2_path, video.fps, crf)

n = len(video)
for i in tqdm(range(n)):
    frame = video[i]
    mask = tracker.track(frame)
    mask = mask.squeeze().detach().cpu().numpy().astype(np.uint8)
    mix_img = generate_mix_image(frame, mask)
    out1.append(mix_img)
    mask_img = generate_mask_image(mask)
    out2.append(mask_img)

out1.close()
out2.close()